In [1]:
import numpy as np
import pandas as pd

### Load ATLAS Data

#### Acceptance

In [2]:
tauDict = {'stau': {0.3 : (9,70), 1.0 : (26,56), 3.0 : (43,42), 10.0 : (60,28), 30.0 : (77,14), 1e10 : (94,0)}
         }
           

dfs = []
files = {'stau' : './HEPData-ins2080541-v1-csv/Acceptance,Stau.csv'}
for tag in tauDict:
    for tau,(skip_header,skip_footer) in tauDict[tag].items():
        data = np.genfromtxt(files[tag],skip_header=skip_header,delimiter=',',names=True,skip_footer=skip_footer)
        newDF = pd.DataFrame.from_dict({'mLLP' : data['Mass_GeV']})
        newDF['Acceptance'] = data['Acceptance']
        newDF['tau_ns'] = tau
        dfs.append(newDF)

atlasAccDF = pd.concat(dfs, ignore_index=True)

#### Event Level Efficiency

In [3]:
dfs = []
files = {'stau' : './HEPData-ins2080541-v1-csv/Event-levelefficiency,Stau.csv'}
for tag in tauDict:
    for tau,(skip_header,skip_footer) in tauDict[tag].items():
        data = np.genfromtxt(files[tag],skip_header=skip_header,delimiter=',',names=True,skip_footer=skip_footer)
        newDF = pd.DataFrame.from_dict({'mLLP' : data['Mass_GeV']})
        newDF['Event Efficiency'] = data['Eventlevel_Efficiency']
        newDF['tau_ns'] = tau
        dfs.append(newDF)

atlasEvtEffDF = pd.concat(dfs, ignore_index=True)

#### SR Efficiency

In [4]:
atlasSRs = []
for sr in ['Inclusive_Low','Inclusive_High']:
    sr_tag = sr.split('_')[1]
    dfs = []
    files = {'stau' : './HEPData-ins2080541-v1-csv/Efficiency,SR-%s,Stau.csv'%sr}
    for tag in tauDict:
        for tau,(skip_header,skip_footer) in tauDict[tag].items():
            data = np.genfromtxt(files[tag],skip_header=skip_header,delimiter=',',names=True,skip_footer=skip_footer)
            newDF = pd.DataFrame.from_dict({'mLLP' : data['Mass_GeV']})
            newDF['(SR-%s - no mass Window)' %sr_tag] = data['Efficiency_%s' %sr]
            newDF['tau_ns'] = tau
            dfs.append(newDF)

    atlasSRs.append(pd.concat(dfs, ignore_index=True))
atlasSREffDF = pd.merge(*atlasSRs)

In [5]:
atlasDF = pd.merge(atlasAccDF, atlasEvtEffDF,how='inner')
atlasDF = pd.merge(atlasDF, atlasSREffDF,how='inner')

In [6]:
orderColumns = ['mLLP','tau_ns']
atlasDF.sort_values(orderColumns,inplace=True,
                ascending=[True,True],ignore_index=True)
res = atlasDF.reindex(columns=orderColumns+[c for c in atlasDF.columns if c not in orderColumns])

In [7]:
atlasDF.to_pickle('stauTables.pcl')

In [8]:
pts = sorted(list(zip(atlasDF['mLLP'],atlasDF['tau_ns'],atlasDF['Acceptance'])))

In [9]:
ptsIni = sorted([("%1.2f" %pt[0], "%1.2g" %float(pt[1])) for pt in pts])
print(str(ptsIni).replace("'", ""))

[(100.00, 0.3), (100.00, 1), (100.00, 10), (100.00, 1e+10), (100.00, 3), (100.00, 30), (1000.00, 0.3), (1000.00, 1), (1000.00, 10), (1000.00, 1e+10), (1000.00, 3), (1000.00, 30), (200.00, 0.3), (200.00, 1), (200.00, 10), (200.00, 1e+10), (200.00, 3), (200.00, 30), (300.00, 0.3), (300.00, 1), (300.00, 10), (300.00, 1e+10), (300.00, 3), (300.00, 30), (400.00, 0.3), (400.00, 1), (400.00, 10), (400.00, 1e+10), (400.00, 3), (400.00, 30), (450.00, 0.3), (450.00, 1), (450.00, 10), (450.00, 1e+10), (450.00, 3), (450.00, 30), (500.00, 0.3), (500.00, 1), (500.00, 10), (500.00, 1e+10), (500.00, 3), (500.00, 30), (550.00, 0.3), (550.00, 1), (550.00, 10), (550.00, 1e+10), (550.00, 3), (550.00, 30), (600.00, 0.3), (600.00, 1), (600.00, 10), (600.00, 1e+10), (600.00, 3), (600.00, 30), (650.00, 0.3), (650.00, 1), (650.00, 10), (650.00, 1e+10), (650.00, 3), (650.00, 30), (700.00, 0.3), (700.00, 1), (700.00, 10), (700.00, 1e+10), (700.00, 3), (700.00, 30), (800.00, 0.3), (800.00, 1), (800.00, 10), (800.